# Comparison of performance of SP and XP

In [1]:
from benchmark_new import *
%matplotlib inline

In [4]:
def highlight_max(x):
    c1 = 'background-color: yellow;'
    c2 = 'font-weight: bold;'
    m1 = x.groupby(level=list(range(len(x.index.names)-3)))['combind_correct'].transform('max').eq(x['combind_correct'])
    m2 = x.groupby(level=list(range(len(x.index.names)-2)))['combind_correct'].transform('max').eq(x['combind_correct'])
    
    df1 = pd.DataFrame('', index=x.index, columns=x.columns)
    df1.loc[m1, 'combind_correct'] += c1
    df1.loc[m2, 'combind_correct'] += c2
    return df1

helpers = 'best_affinity'
mcss = '../../../mcss_sizes.pkl'
data = pd.concat(load('stats{}'.format(version), helpers, mcss)
                 for version in [7, 17, 19, 21])
#data = filter_to_ubiquitous_ligands(data)
data = add_correct(data)
data = data[(data.index.get_level_values('protein') != 'A2AR')]
with pd.ExcelWriter('best_affinity.xlsx', mode='w') as ex:
    for agg in ['drug-target', 'drug-ligand', 'target', 'ligand']:
        for sim, func in [('all', lambda x: x > -1),
                          ('sim', lambda x: x >= 0.5),
                          ('dif', lambda x: x < 0.5)]:
            ligand = data[func(data.mcss)].filter(regex='rmsd|correct')
            target = ligand.groupby(level=list(range(len(ligand.index.levels)-1))).mean().dropna()
            family_target = target.groupby(level=list(range(len(target.index.levels)-1))).mean()
            family_ligand = ligand.groupby(level=list(range(len(ligand.index.levels)-2))).mean()
            
            
            if agg == 'drug-target':
                d = drug_average(family_target)
            elif agg == 'drug-ligand':
                d = drug_average(family_ligand)
            elif agg == 'ligand':
                d = ligand.groupby(level=list(range(len(ligand.index.levels)-3))).mean()
            elif agg == 'target':
                d = target.groupby(level=list(range(len(target.index.levels)-2))).mean()
            
            d = d.filter(regex='correct')
            d = d[d['combind_correct'] != 0]
            styled = d.style.apply(highlight_max, axis = None)
            styled.to_excel(ex, sheet_name='{}-{}'.format(sim, agg))